Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [106]:
from random import random, seed
from itertools import product
import numpy as np

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [107]:
UNIVERSE_SIZE = 10_000
NUM_SETS = 1_000
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [108]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [109]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()


## Have Fun!

In [110]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)

(np.True_, np.float64(4281033.193893639))

In [111]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
valid(solution), cost(solution)

(np.True_, np.float64(2140413.4999994165))

**Greedy algorithm**

In [112]:
def greedy_set_cover():
    solution = np.full(NUM_SETS, False)  # Start with no sets selected
    uncovered_elements = np.where(np.any(SETS, axis=0))[0]  # Elements still uncovered
    iteration = 0

    while not valid(solution):
        best_set_index = None
        max_uncovered = 0
        best_set_cost = cost(np.full(NUM_SETS, True))

        # Look for the best set to add
        for set_index in range(NUM_SETS):
            if not solution[set_index]:
                uncovered_in_set = np.sum(SETS[set_index] & ~np.logical_or.reduce(SETS[solution]))
                if uncovered_in_set > max_uncovered or (uncovered_in_set == max_uncovered and COSTS[set_index] < best_set_cost):
                    max_uncovered = uncovered_in_set
                    best_set_index = set_index
                    best_set_cost = COSTS[set_index]

        # Add the best set to the solution
        if best_set_index is not None:
            solution[best_set_index] = True
            uncovered_elements = np.setdiff1d(uncovered_elements, np.where(SETS[best_set_index])[0])


        ic(iteration, valid(solution), cost(solution), len(np.where(solution)[0]))
        iteration += 1

    return solution, cost(solution)

# Run the greedy algorithm
greedy_solution, greedy_cost = greedy_set_cover()
print(f"Greedy solution cost: {greedy_cost}")


ic| iteration: 0
    valid(solution): np.False_
    cost(solution): np.float64(4586.081777779552)
    len(np.where(solution)[0]): 1
ic| iteration: 1
    valid(solution): np.False_
    cost(solution): np.float64(9155.595266441425)
    len(np.where(solution)[0]): 2
ic| iteration: 2
    valid(solution): np.False_
    cost(solution): np.float64(13665.980975003891)
    len(np.where(solution)[0]): 3
ic| iteration: 3
    valid(solution): np.False_
    cost(solution): np.float64(18088.986960062353)
    len(np.where(solution)[0]): 4
ic| iteration: 4
    valid(solution): np.False_
    cost(solution): np.float64(22587.55553444305)
    len(np.where(solution)[0]): 5
ic| iteration: 5
    valid(solution): np.False_
    cost(solution): np.float64(27048.328268773814)
    len(np.where(solution)[0]): 6
ic| iteration: 6
    valid(solution): np.False_
    cost(solution): np.float64(31351.149250104198)
    len(np.where(solution)[0]): 7
ic| iteration: 7
    valid(solution): np.False_
    cost(solution): np.f

Greedy solution cost: 100157.39431320036


*function for hill climbing*

In [113]:
def fitness(solution):
    """Returns the fitness as -cost, penalizing invalid solutions"""
    return -cost(solution)


# Single mutation tweak (switches one set)
def single_tweak(solution):
    new_solution = solution.copy()
    i = rng.integers(0, NUM_SETS)
    new_solution[i] = not new_solution[i]
    return new_solution

# Multiple tweaks (randomly flips multiple sets)
def multiple_tweak(solution, prob=0.01):
    mask = rng.random(NUM_SETS) < prob
    new_solution = np.logical_xor(solution, mask)
    return new_solution

**Hill Climbing algorithm single tweak**

In [114]:
# Hill Climbing algorithm single tweak
def hill_climbing(tweak_function, max_iters=1000):
    solution = rng.random(NUM_SETS) < 0.5  # Start with 50% sets selected
    best_fitness = fitness(solution)

    for iteration in range(max_iters):
        new_solution = tweak_function(solution)
        new_fitness = fitness(new_solution)

        if new_fitness > best_fitness:  # If the new solution is better
            solution = new_solution
            best_fitness = new_fitness

        ic(iteration, valid(solution), cost(solution), len(np.where(solution)[0]))

    return solution, best_fitness

# Running the hill climbing algorithm with single tweak
best_solution, best_fitness = hill_climbing(single_tweak, max_iters=1000)
print(f"Best solution (multiple tweak) cost: {-best_fitness}")


ic| iteration: 0
    valid(solution): np.True_
    cost(solution): np.float64(2115481.788239298)
    len(np.where(solution)[0]): 494
ic| iteration: 1
    valid(solution): np.True_
    cost(solution): np.float64(2115481.788239298)
    len(np.where(solution)[0]): 494
ic| iteration: 2
    valid(solution): np.True_
    cost(solution): np.float64(2111110.6636611433)
    len(np.where(solution)[0]): 493
ic| iteration: 3
    valid(solution): np.True_
    cost(solution): np.float64(2111110.6636611433)
    len(np.where(solution)[0]): 493
ic| iteration: 4
    valid(solution): np.True_
    cost(solution): np.float64(2111110.6636611433)
    len(np.where(solution)[0]): 493
ic| iteration: 5
    valid(solution): np.True_
    cost(solution): np.float64(2111110.6636611433)
    len(np.where(solution)[0]): 493
ic| iteration: 6
    valid(solution): np.True_
    cost(solution): np.float64(2106826.6677837856)
    len(np.where(solution)[0]): 492
ic| iteration: 7
    valid(solution): np.True_
    cost(solution

Best solution (multiple tweak) cost: 753551.16732018


**Hill Climbing algorithm with multiple tweaks**

In [115]:
# Hill Climbing algorithm with multiple tweaks 
def hill_climbing(tweak_function, max_iters=1000):
    solution = rng.random(NUM_SETS) < 0.5  # Start with 50% sets selected
    best_fitness = fitness(solution)

    for iteration in range(max_iters):
        new_solution = tweak_function(solution)
        new_fitness = fitness(new_solution)

        if new_fitness > best_fitness:  # If the new solution is better
            solution = new_solution
            best_fitness = new_fitness

        ic(iteration, valid(solution), cost(solution), len(np.where(solution)[0]), best_fitness)

    return solution, best_fitness

# Running the hill climbing algorithm with multiple tweaks
best_solution_multi, best_fitness_multi = hill_climbing(lambda sol: multiple_tweak(sol, prob=0.01), max_iters=1000)
print(f"Best solution (multiple tweak) cost: {-best_fitness_multi}")


ic| iteration: 0
    valid(solution): np.True_
    cost(solution): np.float64(2249757.3060013438)
    len(np.where(solution)[0]): 526
    best_fitness: np.float64(-2249757.3060013438)
ic| iteration: 1
    valid(solution): np.True_
    cost(solution): np.float64(2249757.3060013438)
    len(np.where(solution)[0]): 526
    best_fitness: np.float64(-2249757.3060013438)
ic| iteration: 2
    valid(solution): np.True_
    cost(solution): np.float64(2249757.3060013438)
    len(np.where(solution)[0]): 526
    best_fitness: np.float64(-2249757.3060013438)
ic| iteration: 3
    valid(solution): np.True_
    cost(solution): np.float64(2249757.3060013438)
    len(np.where(solution)[0]): 526
    best_fitness: np.float64(-2249757.3060013438)
ic| iteration: 4
    valid(solution): np.True_
    cost(solution): np.float64(2232592.9262890485)
    len(np.where(solution)[0]): 522
    best_fitness: np.float64(-2232592.9262890485)
ic| iteration: 5
    valid(solution): np.True_
    cost(solution): np.float64(22

Best solution (multiple tweak) cost: 1110838.6733144075


**Tweak and Restart**

In [116]:
# Tweak and Restart algorithm
def tweak_and_restart(max_restarts=100):
    best_solution = np.full(NUM_SETS, True)  # Initial "best" solution with all sets
    
    for restart in range(max_restarts):
        # Start with no sets
        solution = np.full(NUM_SETS, False)

        # Randomly add sets until the solution is valid
        while not valid(solution):
            index = rng.integers(0, NUM_SETS)
            solution[index] = True
        
        # Check if the new solution has a lower cost
        if cost(solution) < cost(best_solution):
            best_solution = solution.copy()

        ic(restart, valid(solution), cost(solution), len(np.where(solution)[0]), cost(best_solution))

    return best_solution, cost(best_solution)

# Run the tweak and restart algorithm
best_solution, best_cost = tweak_and_restart(max_restarts=100)
print(f"Best solution found with tweak and restart: {best_cost}")


ic| restart: 0
    valid(solution): np.True_
    cost(solution): np.float64(183369.30966859363)
    len(np.where(solution)[0]): 43
    cost(best_solution): np.float64(183369.30966859363)
ic| restart: 1
    valid(solution): np.True_
    cost(solution): np.float64(157881.10944030635)
    len(np.where(solution)[0]): 37
    cost(best_solution): np.float64(157881.10944030635)
ic| restart: 2
    valid(solution): np.True_
    cost(solution): np.float64(157833.16821861753)
    len(np.where(solution)[0]): 37
    cost(best_solution): np.float64(157833.16821861753)
ic| restart: 3
    valid(solution): np.True_
    cost(solution): np.float64(227258.50401691897)
    len(np.where(solution)[0]): 53
    cost(best_solution): np.float64(157833.16821861753)
ic| restart: 4
    valid(solution): np.True_
    cost(solution): np.float64(201126.99484993736)
    len(np.where(solution)[0]): 47
    cost(best_solution): np.float64(157833.16821861753)
ic| restart: 5
    valid(solution): np.True_
    cost(solution): 

Best solution found with tweak and restart: 157833.16821861753
